AVITO ONE WEEKEND OFFER

In [2]:
import pandas as pd

In [3]:
data = pd.read_parquet('mlsd-eda.parquet', engine='pyarrow')

In [4]:
data.head()

,event_date,eid,x_eid,user_id,item_id,microcat_id
0,2023-04-16 23:59:59,301,300,2010890,1435852750630,3674
1,2023-04-16 23:59:59,301,300,379294501,1437733502707,1673
2,2023-04-16 23:59:59,301,300,198428250055,1444440250530,3831
3,2023-04-16 23:59:59,301,2012,705812500004,1444999750230,347447750001
4,2023-04-16 23:59:59,301,2012,1116509750025,1378823251390,1804


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19083067 entries, 0 to 19083066
Data columns (total 6 columns):
 #   Column       Dtype         
---  ------       -----         
 0   event_date   datetime64[us]
 1   eid          int64         
 2   x_eid        int64         
 3   user_id      int64         
 4   item_id      int64         
 5   microcat_id  int64         
dtypes: datetime64[us](1), int64(5)
memory usage: 873.6 MB


In [5]:
# Вывод всех уникальных значений для столбца eid
unique_eid = data['eid'].unique()
print("Уникальные значения eid:", unique_eid)

# Вывод всех уникальных значений для столбца x_eid
unique_x_eid = data['x_eid'].unique()
print("Уникальные значения x_eid:", unique_x_eid)

Уникальные значения eid: [301 401 857 303 308]
Уникальные значения x_eid: [ 300 2012 2437 2017  400]


Хочу посмотреть с какими x_eid идет eid = 300

In [8]:
eid_308_data = data[data['eid'] == 308]
eid_308_x_eid_counts = eid_308_data.groupby('x_eid').size().reset_index(name='count')
print(eid_308_x_eid_counts)

   x_eid  count
0    300  66182
1    400  31845
2   2012  92532
3   2017   6786
4   2437   8619


Чему соответсвует eid = 308 и с какого города данные???

eid (id события)

- 301  просмотр
- 303  просмотр номера телефона (является контактом)
- 401  добавление в избранное
- 857  отправка сообщения (является контактом)

x_eid (id источника, откуда произошло событие)

- 300  поисковая выдача
- 400  список избранного
- 2012  главная страница (рекомендации)
- 2017  профиль продавца
- 2437  похожие объявления на карточке товара "


In [9]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19083067 entries, 0 to 19083066
Data columns (total 6 columns):
 #   Column       Dtype         
---  ------       -----         
 0   event_date   datetime64[us]
 1   eid          int64         
 2   x_eid        int64         
 3   user_id      int64         
 4   item_id      int64         
 5   microcat_id  int64         
dtypes: datetime64[us](1), int64(5)
memory usage: 873.6 MB


In [10]:
data.head(1)

,event_date,eid,x_eid,user_id,item_id,microcat_id
0,2023-04-16 23:59:59,301,300,2010890,1435852750630,3674


1. Какое самое большое число различных объявлений пользователь добавил в избранное (событие 401) за день?

In [11]:
# Отфильтруем данные по событию 401 (добавление в избранное)
favorites_data = data[data['eid'] == 401]

# Преобразуем 'event_date' в формат только даты (игнорируем время)
favorites_data['event_date_day'] = favorites_data['event_date'].dt.date

# Группируем данные по пользователям и дням, считаем количество уникальных item_id (объявлений)
favorites_per_day = favorites_data.groupby(['user_id', 'event_date_day'])['item_id'].nunique()

# находим максимум
max_favorites_per_day = favorites_per_day.max()
max_favorites_per_day

<ipython-input-11-b090380793f4>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  favorites_data['event_date_day'] = favorites_data['event_date'].dt.date


833

----

2. Сколько пользователей чаще пользуются рекомендациями (источники действий - 2012 и 2437), чем поиском (300)? В ответ напишите целое число - количество пользователей.

In [12]:
recommendation_sources = data[data['x_eid'].isin([2012, 2437])]
search_source = data[data['x_eid'] == 300]

print(recommendation_sources.shape)
print(search_source.shape)

(11276629, 6)
(5886128, 6)


In [26]:
# 1. Подсчитываем количество действий по рекомендациям для каждого пользователя
recommendation_counts = recommendation_sources.groupby('user_id').size().reset_index(name='recommendation_count')

# 2. Подсчитываем количество действий по поиску для каждого пользователя
search_counts = search_source.groupby('user_id').size().reset_index(name='search_count')

In [28]:
# 3. Объединяем данные для пользователей, чтобы сравнить действия по рекомендациям и поиску
merged_counts = pd.merge(recommendation_counts, search_counts, on='user_id', how='outer').fillna(0)
merged_counts

,user_id,recommendation_count,search_count
0,196,1.0,0.0
1,216,3.0,0.0
2,299,2.0,0.0
3,412,3.0,8.0
4,453,1.0,0.0
...,...,...,...
1764111,1196041500038,0.0,2.0
1764112,1196041750010,0.0,1.0
1764113,1196043000038,0.0,1.0
1764114,1196048500154,0.0,1.0


In [29]:
# 4. Оставляем только тех пользователей, у которых количество действий в рекомендациях больше, чем в поиске
users_more_recommendations = merged_counts[merged_counts['recommendation_count'] > merged_counts['search_count']]
users_more_recommendations

,user_id,recommendation_count,search_count
0,196,1.0,0.0
1,216,3.0,0.0
2,299,2.0,0.0
4,453,1.0,0.0
5,680,2.0,1.0
...,...,...,...
958431,1196033500086,2.0,0.0
958432,1196033500228,3.0,0.0
958433,1196033500296,39.0,8.0
958434,1196037750206,2.0,0.0


In [30]:
# 5. Подсчитываем количество таких пользователей
num_users = users_more_recommendations.shape[0]
num_users

750925

----

1. Какая доля действий в приходится на самое популярное (объявление с наибольшим количеством действий) объявление в его микрокатегории? Напишите ответ от 0 до 1 с точностью до 3 знака после запятой. Пример: у самого популярного объявления 111 действий, а в его мирокатегории - 1000. Значит ответ должен быть 0.111


- 1
- 0.924
- 0.186
- 0.003


In [43]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19083067 entries, 0 to 19083066
Data columns (total 6 columns):
 #   Column       Dtype         
---  ------       -----         
 0   event_date   datetime64[us]
 1   eid          int64         
 2   x_eid        int64         
 3   user_id      int64         
 4   item_id      int64         
 5   microcat_id  int64         
dtypes: datetime64[us](1), int64(5)
memory usage: 873.6 MB


In [44]:
data.head(2)

,event_date,eid,x_eid,user_id,item_id,microcat_id
0,2023-04-16 23:59:59,301,300,2010890,1435852750630,3674
1,2023-04-16 23:59:59,301,300,379294501,1437733502707,1673


In [34]:
# Считаем количество действий для каждого объявления
actions_per_item = data.groupby(['microcat_id', 'item_id']).size().reset_index(name='action_count')

# Находим самое популярное объявление (с максимальным количеством действий)
most_popular_item = actions_per_item.loc[actions_per_item['action_count'].idxmax()]

# Находим микрокатегорию самого популярного объявления
most_popular_microcat = most_popular_item['microcat_id']

# Считаем общее количество действий в микрокатегории этого объявления
total_actions_in_microcat = data[data['microcat_id'] == most_popular_microcat].shape[0]

# Рассчитываем долю действий самого популярного объявления в микрокатегории
action_ratio = most_popular_item['action_count'] / total_actions_in_microcat

# Выводим результаты
print(f"Самое популярное объявление:\n{most_popular_item}")
print('='*50)
print(f"Общее количество действий в микрокатегории: {total_actions_in_microcat}")
print(f"Доля действий данного объявления в микрокатегории: {action_ratio:.3f}")

Самое популярное объявление:
microcat_id           3250006
item_id         1433680000132
action_count            57097
Name: 683120, dtype: int64
Общее количество действий в микрокатегории: 61809
Доля действий данного объявления в микрокатегории: 0.924


2. Какая микрокатегория наиболее похожа на микрокатегорию 259 (кроме неё самой)? Похожесть определяется так: количество пользователей, которые смотрели исходную и целевую мирокатегорию вместе (за всё время). В ответ запишите microcat_id.


- 6702
- 266425750001
- 5927
- 149

In [37]:
# Выбираем всех пользователей, которые смотрели микрокатегорию 259
users_in_microcat_259 = set(data[data['microcat_id'] == 259]['user_id'])
len(users_in_microcat_259) # set

12941

In [39]:
# Считаем пользователей для каждой микрокатегории, кроме 259
other_microcats = data[data['microcat_id'] != 259]
len(other_microcats) # df

18966231

In [44]:
# Группируем по микрокатегориям и создаем множества пользователей для каждой микрокатегории
user_sets_per_microcat = other_microcats.groupby('microcat_id')['user_id'].apply(set).reset_index()
user_sets_per_microcat

,microcat_id,user_id
0,4,"{637649750017, 1032718500035, 377281667, 10561..."
1,5,"{1087381250049, 1084459500035, 561013250051, 5..."
2,10,"{104997250051, 179401750022, 1121797250059, 14..."
3,11,"{679360000010, 871075500044, 784832000012, 487..."
4,15,"{218211500032, 1147640750082, 280647000070, 10..."
...,...,...
2626,394327750001,"{444167000064, 307327000066, 248200000021, 852..."
2627,394329000001,"{378001750017, 631987750017, 921432000004, 375..."
2628,394329000002,{126466250053}
2629,394330000001,"{358974750103, 1022607500057, 797159250083, 41..."


In [45]:
# Вычисляем количество пересечений пользователей для каждой микрокатегории с микрокатегорией 259
user_sets_per_microcat['intersection_size'] = user_sets_per_microcat['user_id'].apply(lambda x: len(x & users_in_microcat_259))

user_sets_per_microcat

,microcat_id,user_id,intersection_size
0,4,"{637649750017, 1032718500035, 377281667, 10561...",1
1,5,"{1087381250049, 1084459500035, 561013250051, 5...",4
2,10,"{104997250051, 179401750022, 1121797250059, 14...",4
3,11,"{679360000010, 871075500044, 784832000012, 487...",7
4,15,"{218211500032, 1147640750082, 280647000070, 10...",100
...,...,...,...
2626,394327750001,"{444167000064, 307327000066, 248200000021, 852...",3
2627,394329000001,"{378001750017, 631987750017, 921432000004, 375...",0
2628,394329000002,{126466250053},0
2629,394330000001,"{358974750103, 1022607500057, 797159250083, 41...",0


In [49]:
# Находим микрокатегорию с наибольшим количеством пересечений
most_similar_microcat = user_sets_per_microcat.loc[user_sets_per_microcat['intersection_size'].idxmax()]
most_similar_microcat


,884
microcat_id,6702
user_id,"{827313750016, 72469250048, 349226500098, 1870..."
intersection_size,3352


In [51]:
# Выводим ID самой похожей микрокатегории
print(f"Микрокатегория, наиболее похожая на 259: {most_similar_microcat['microcat_id']}")

Микрокатегория, наиболее похожая на 259: 6702


----

3. В какой день недели доля контактов (303 и 857) с главной страницы (2012) от всех контактов наибольшая?

понедельник
вторник
среда
четверг
пятница
суббота
воскресенье

In [69]:
# Фильтруем данные по событиям, которые связаны с контактами (303 и 857)
contacts = data[data['eid'].isin([303, 857])]
# Фильтруем контакты, которые произошли с главной страницы (источник 2012)
contacts_from_main = contacts[contacts['x_eid'] == 2012]


In [70]:
# Добавляем столбец с днем недели для каждого события (0 = понедельник, 6 = воскресенье)
contacts['weekday'] = contacts['event_date'].dt.dayofweek
contacts_from_main['weekday'] = contacts_from_main['event_date'].dt.dayofweek

<ipython-input-70-cbb37c7c7eab>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  contacts['weekday'] = contacts['event_date'].dt.dayofweek
<ipython-input-70-cbb37c7c7eab>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  contacts_from_main['weekday'] = contacts_from_main['event_date'].dt.dayofweek


In [71]:
# Считаем количество контактов по каждому дню недели
total_contacts_per_day = contacts.groupby('weekday').size().reset_index(name='total_contacts')
# Считаем количество контактов с главной страницы по каждому дню недели
contacts_from_main_per_day = contacts_from_main.groupby('weekday').size().reset_index(name='contacts_from_main')

In [72]:
# Объединяем данные для расчета доли
merged = pd.merge(contacts_from_main_per_day, total_contacts_per_day, on='weekday')
# Рассчитываем долю контактов с главной страницы от всех контактов по каждому дню
merged['contact_ratio'] = merged['contacts_from_main'] / merged['total_contacts']

# Находим день недели с наибольшей долей
best_day = merged.loc[merged['contact_ratio'].idxmax()]

In [73]:
best_day

,6
weekday,6.000000
contacts_from_main,11010.000000
total_contacts,45047.000000
contact_ratio,0.244411


In [74]:
# Преобразуем индекс в название дня недели
days_of_week = ['понедельник', 'вторник', 'среда', 'четверг', 'пятница', 'суббота', 'воскресенье']

# Выводим итог
print(f"День недели с наибольшей долей контактов с главной страницы: {days_of_week[int(best_day['weekday'])]}")
print(f"Доля контактов: {best_day['contact_ratio']:.3f}")


День недели с наибольшей долей контактов с главной страницы: воскресенье
Доля контактов: 0.244


#### ИЛИ ТАК

In [ ]:

import pandas as pd

# 1. Фильтруем данные по событиям, которые связаны с контактами (eid == 303 или eid == 857)
contacts = data[data['eid'].isin([303, 857])]

# 2. Фильтруем контакты, которые произошли с главной страницы (x_eid == 2012)
contacts_from_main = contacts[contacts['x_eid'] == 2012]

# 3. Добавляем столбец с днем недели для каждого события (0 = понедельник, 6 = воскресенье)
data['weekday'] = data['event_date'].dt.dayofweek
contacts_from_main['weekday'] = contacts_from_main['event_date'].dt.dayofweek

# 4. Считаем общее количество событий по каждому дню недели (учитываем все события, независимо от eid и x_eid)
total_events_per_day = data.groupby('weekday').size().reset_index(name='total_events')

# 5. Считаем количество контактов с главной страницы по каждому дню недели (с фильтром по x_eid == 2012)
contacts_from_main_per_day = contacts_from_main.groupby('weekday').size().reset_index(name='contacts_from_main')

# 6. Объединяем данные для расчета доли
merged = pd.merge(contacts_from_main_per_day, total_events_per_day, on='weekday')

# 7. Рассчитываем долю контактов с главной страницы от всех событий по каждому дню
merged['contact_ratio'] = merged['contacts_from_main'] / merged['total_events']

# 8. Находим день недели с наибольшей долей
best_day = merged.loc[merged['contact_ratio'].idxmax()]

# Преобразуем индекс в название дня недели
days_of_week = ['понедельник', 'вторник', 'среда', 'четверг', 'пятница', 'суббота', 'воскресенье']

# Выводим итог
print(f"День недели с наибольшей долей контактов с главной страницы: {days_of_week[int(best_day['weekday'])]}")
print(f"Доля контактов: {best_day['contact_ratio']:.3f}")


4. Сколько минут длилась самая длинная по времени сессия? Сессия - такая последовательность событий одного пользователя, что между двумя соседними по времени событиями разница не более 10 минут. Длина сессии - разность между первым и последним событием.

- 10078
- 308
- 218
- 10


In [75]:
# Сортируем данные по user_id и времени события
data_sorted = data.sort_values(by=['user_id', 'event_date'])
data_sorted.head(20)

,event_date,eid,x_eid,user_id,item_id,microcat_id
16158814,2023-04-10 20:20:23,301,300,8,1387716002217,299603000002
15906413,2023-04-11 00:26:36,301,300,8,1442728502422,338648500001
15905735,2023-04-11 00:27:53,857,300,8,1442728502422,338648500001
15905673,2023-04-11 00:28:02,401,300,8,1442728502422,338648500001
15845429,2023-04-11 02:25:55,301,300,8,1091515500305,338654500001
2065302,2023-04-16 09:25:44,301,300,69,1269829000410,266425750001
1667723,2023-04-16 11:57:30,301,300,69,1377979500564,266425750001
993275,2023-04-16 15:59:34,301,300,69,1269829000410,266425750001
426606,2023-04-16 18:59:40,301,300,115,1444171750114,4044
14929678,2023-04-11 09:43:25,301,300,134,1438974750459,3975


In [76]:
# Рассчитываем разницу во времени между соседними событиями (в минутах)
data_sorted['time_diff'] = data_sorted.groupby('user_id')['event_date'].diff().dt.total_seconds() / 60
data_sorted.head(20)

,event_date,eid,x_eid,user_id,item_id,microcat_id,time_diff
16158814,2023-04-10 20:20:23,301,300,8,1387716002217,299603000002,NaN
15906413,2023-04-11 00:26:36,301,300,8,1442728502422,338648500001,246.216667
15905735,2023-04-11 00:27:53,857,300,8,1442728502422,338648500001,1.283333
15905673,2023-04-11 00:28:02,401,300,8,1442728502422,338648500001,0.150000
15845429,2023-04-11 02:25:55,301,300,8,1091515500305,338654500001,117.883333
2065302,2023-04-16 09:25:44,301,300,69,1269829000410,266425750001,NaN
1667723,2023-04-16 11:57:30,301,300,69,1377979500564,266425750001,151.766667
993275,2023-04-16 15:59:34,301,300,69,1269829000410,266425750001,242.066667
426606,2023-04-16 18:59:40,301,300,115,1444171750114,4044,NaN
14929678,2023-04-11 09:43:25,301,300,134,1438974750459,3975,NaN


In [77]:
# Если разница больше 10 минут, то это начало новой сессии, обозначаем её
data_sorted['new_session'] = (data_sorted['time_diff'] > 10).fillna(False).astype(int)

# Определяем номер сессии для каждого пользователя
data_sorted['session_id'] = data_sorted.groupby('user_id')['new_session'].cumsum()

# Находим время начала и конца каждой сессии
session_times = data_sorted.groupby(['user_id', 'session_id']).agg(
    session_start=('event_date', 'min'),
    session_end=('event_date', 'max')
).reset_index()

# Рассчитываем длительность каждой сессии в минутах
session_times['session_duration'] = (session_times['session_end'] - session_times['session_start']).dt.total_seconds() / 60

# Находим максимальную продолжительность сессии
longest_session = session_times['session_duration'].max()

# Выводим результат
print(f"Самая длинная сессия длилась {longest_session:.2f} минут.")

Самая длинная сессия длилась 308.25 минут.


----

5. Какая доля объявлений имеет только просмотры (301 событие)? Напишите ответ от 0 до 1 с точностью до 3 знака после запятой.

0.61
0.604
0.994
0.463

In [71]:
# Группируем данные по объявлениям и считаем уникальные события для каждого объявления
events_per_item = data.groupby('item_id')['eid'].unique().reset_index()

# Фильтруем объявления, у которых только одно уникальное событие, и это событие равно 301
only_views = events_per_item[events_per_item['eid'].apply(lambda x: set(x) == {301})]

# Считаем долю объявлений, у которых только просмотры
total_items = data['item_id'].nunique()
only_views_count = only_views['item_id'].nunique()

# Рассчитываем долю
view_only_ratio = only_views_count / total_items

# Выводим результат с точностью до 3 знаков после запятой
print(f"Доля объявлений, у которых только просмотры: {view_only_ratio:.3f}")


Доля объявлений, у которых только просмотры: 0.604


----

6. Какая доля от всех просмотров (301) является повторными просмотрами? Повторный просмотр - это когда пользователь ранее смотрел это же объявление, и с момента хотя бы одного просмотра этого объявления прошло не менее часа. Напишите ответ от 0 до 1 с точностью до 3 знака после запятой.

- 0.079
- 0.157
- 0.062
- 0.071

In [80]:
views = data[data['eid'] == 301]
views.head(10)

,event_date,eid,x_eid,user_id,item_id,microcat_id
0,2023-04-16 23:59:59,301,300,2010890,1435852750630,3674
1,2023-04-16 23:59:59,301,300,379294501,1437733502707,1673
2,2023-04-16 23:59:59,301,300,198428250055,1444440250530,3831
3,2023-04-16 23:59:59,301,2012,705812500004,1444999750230,347447750001
4,2023-04-16 23:59:59,301,2012,1116509750025,1378823251390,1804
6,2023-04-16 23:59:58,301,300,82953500002,1330956750618,347447750001
7,2023-04-16 23:59:58,301,300,386596750293,1343212502692,4840
8,2023-04-16 23:59:58,301,300,947673500005,1442321000806,266425750001
9,2023-04-16 23:59:58,301,2012,368098039,1440974001837,504332
10,2023-04-16 23:59:58,301,2012,448620500157,1381372000782,372699000001


In [81]:
views = views.sort_values(by=['user_id', 'item_id', 'event_date'])
views.head(10)

,event_date,eid,x_eid,user_id,item_id,microcat_id
15845429,2023-04-11 02:25:55,301,300,8,1091515500305,338654500001
16158814,2023-04-10 20:20:23,301,300,8,1387716002217,299603000002
15906413,2023-04-11 00:26:36,301,300,8,1442728502422,338648500001
2065302,2023-04-16 09:25:44,301,300,69,1269829000410,266425750001
993275,2023-04-16 15:59:34,301,300,69,1269829000410,266425750001
1667723,2023-04-16 11:57:30,301,300,69,1377979500564,266425750001
426606,2023-04-16 18:59:40,301,300,115,1444171750114,4044
14859173,2023-04-11 10:10:52,301,300,134,1310952250087,4375
14929678,2023-04-11 09:43:25,301,300,134,1438974750459,3975
6860989,2023-04-14 10:22:39,301,300,134,1438974750459,3975


In [82]:
views['time_diff'] = views.groupby(['user_id', 'item_id'])['event_date'].diff().dt.total_seconds() / 3600  # разница в часах
views.head(10)

,event_date,eid,x_eid,user_id,item_id,microcat_id,time_diff
15845429,2023-04-11 02:25:55,301,300,8,1091515500305,338654500001,NaN
16158814,2023-04-10 20:20:23,301,300,8,1387716002217,299603000002,NaN
15906413,2023-04-11 00:26:36,301,300,8,1442728502422,338648500001,NaN
2065302,2023-04-16 09:25:44,301,300,69,1269829000410,266425750001,NaN
993275,2023-04-16 15:59:34,301,300,69,1269829000410,266425750001,6.563889
1667723,2023-04-16 11:57:30,301,300,69,1377979500564,266425750001,NaN
426606,2023-04-16 18:59:40,301,300,115,1444171750114,4044,NaN
14859173,2023-04-11 10:10:52,301,300,134,1310952250087,4375,NaN
14929678,2023-04-11 09:43:25,301,300,134,1438974750459,3975,NaN
6860989,2023-04-14 10:22:39,301,300,134,1438974750459,3975,72.653889


In [83]:
# Считаем просмотр повторным, если разница во времени больше или равна 1 часу
views['is_repeat'] = views['time_diff'] >= 1
views.head(10)

,event_date,eid,x_eid,user_id,item_id,microcat_id,time_diff,is_repeat
15845429,2023-04-11 02:25:55,301,300,8,1091515500305,338654500001,NaN,False
16158814,2023-04-10 20:20:23,301,300,8,1387716002217,299603000002,NaN,False
15906413,2023-04-11 00:26:36,301,300,8,1442728502422,338648500001,NaN,False
2065302,2023-04-16 09:25:44,301,300,69,1269829000410,266425750001,NaN,False
993275,2023-04-16 15:59:34,301,300,69,1269829000410,266425750001,6.563889,True
1667723,2023-04-16 11:57:30,301,300,69,1377979500564,266425750001,NaN,False
426606,2023-04-16 18:59:40,301,300,115,1444171750114,4044,NaN,False
14859173,2023-04-11 10:10:52,301,300,134,1310952250087,4375,NaN,False
14929678,2023-04-11 09:43:25,301,300,134,1438974750459,3975,NaN,False
6860989,2023-04-14 10:22:39,301,300,134,1438974750459,3975,72.653889,True


In [84]:
# Подсчитываем общее количество просмотров и количество повторных просмотров
total_views = views.shape[0]
repeat_views = views['is_repeat'].sum()

# Рассчитываем долю повторных просмотров
repeat_view_ratio = repeat_views / total_views

# Выводим результат с точностью до 3 знаков после запятой
print(f"Доля повторных просмотров: {repeat_view_ratio:.3f}")


Доля повторных просмотров: 0.062


----

7 Обучим модель предсказания retention - понять, придет ли к нам пользователь в день Х на основе его действий в предыдущие дни.

Для этого сделаем следующее:
1. Разобьем пользователей на train и test по принципу train: user_id % 11 == 1, test: user_id % 11 == 0
2. Сформируем бинарный таргет: 1, если пользователь совершал действия в последний день (2023-04-16), 0 - иначе
3. Построим признаки - счетчики по всем событиям и источникам, например count_301_300 (просмотры с поиска) (для построения исключаем день, который предсказываем). Должно получиться 25 признаков.
4. Обучим модель sklearn.KNearestNeigborsClassifier с n_neighbors=5, предскажем вероятности классов для тестовой выборки.
5. Измерим roc_auc_score на тестовой выборке, это и есть ответ на это задание, округлить до 3 знака

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import roc_auc_score

In [2]:

data = pd.read_parquet('mlsd-eda.parquet', engine='pyarrow')

# 1. Разделение на train и test
data['train'] = data['user_id'] % 11 == 1
data['test'] = data['user_id'] % 11 == 0

In [3]:
# 2. Формирование бинарного таргета
last_day = data['event_date'].max().date()
# Добавляем колонку с датой без времени
data['event_date_day'] = data['event_date'].dt.date
# Формируем таргет: 1, если пользователь был активен в последний день
data['target'] = (data['event_date_day'] == last_day).astype(int)

In [7]:
data['target'].value_counts()

,count
target,
0,16150824
1,2932243


In [14]:
# 3. Построение признаков

# Фильтруем данные за все дни, кроме дня предсказания
train_data = data[data['event_date_day'] < last_day]

# Создаем счетчики по комбинациям событий и источников
pivot = train_data.pivot_table(index='user_id', columns=['eid', 'x_eid'], aggfunc='size', fill_value=0)

# Приводим MultiIndex к обычным столбцам
pivot.columns = [f'count_{eid}_{x_eid}' for eid, x_eid in pivot.columns]

pivot = pivot.reset_index()


In [17]:
data.shape

(19083067, 10)

In [18]:
# 4. Объединение данных с бинарным таргетом
target_data = data[['user_id', 'target', 'train', 'test']].drop_duplicates()

# Объединяем таргет с признаками
final_data = pd.merge(pivot, target_data, on='user_id', how='left')

# Заполняем отсутствующие значения нулями (если у пользователей не было активности)
final_data.fillna(0, inplace=True)

In [20]:
# Отделим тренировочные и тестовые данные
train_features = final_data[final_data['train'] == True]
test_features = final_data[final_data['test'] == True]

In [21]:
train_features

,user_id,count_301_300,count_301_400,count_301_2012,count_301_2017,count_301_2437,count_303_300,count_303_400,count_303_2012,count_303_2017,...,count_401_2017,count_401_2437,count_857_300,count_857_400,count_857_2012,count_857_2017,count_857_2437,target,train,test
3,144,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,True,False
22,727,0,0,1,0,2,0,0,0,0,...,0,0,0,0,0,0,0,0,True,False
75,2245,3,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,True,False
80,2344,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,True,False
113,2861,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1906836,1195569750013,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,True,False
1906867,1195581250062,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,True,False
1906874,1195585500066,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,True,False
1906882,1195597500010,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,True,False


In [22]:
# 5. Разделение на train и test наборы

X_train = train_features.drop(columns=['target', 'user_id', 'train', 'test'], axis=1)
y_train = train_features['target']

X_test = test_features.drop(columns=['target', 'user_id', 'train', 'test'], axis=1)
y_test = test_features['target']

In [23]:
# Обучение модели
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train, y_train)


KNeighborsClassifier()

In [24]:
# Предсказание вероятностей для тестовой выборки
y_pred_proba = knn.predict_proba(X_test)[:, 1]

# Оценка качества модели с помощью roc_auc_score
roc_auc = roc_auc_score(y_test, y_pred_proba)

# Вывод результата с точностью до 3 знаков
print(f"ROC AUC Score: {roc_auc:.3f}")


ROC AUC Score: 0.669


In [10]:
data.head(20)

,event_date,eid,x_eid,user_id,item_id,microcat_id,train,test,event_date_day,target
0,2023-04-16 23:59:59,301,300,2010890,1435852750630,3674,False,False,2023-04-16,1
1,2023-04-16 23:59:59,301,300,379294501,1437733502707,1673,False,False,2023-04-16,1
2,2023-04-16 23:59:59,301,300,198428250055,1444440250530,3831,False,False,2023-04-16,1
3,2023-04-16 23:59:59,301,2012,705812500004,1444999750230,347447750001,False,False,2023-04-16,1
4,2023-04-16 23:59:59,301,2012,1116509750025,1378823251390,1804,False,False,2023-04-16,1
5,2023-04-16 23:59:59,401,2012,1120685000011,1412604252592,4400,False,False,2023-04-16,1
6,2023-04-16 23:59:58,301,300,82953500002,1330956750618,347447750001,False,False,2023-04-16,1
7,2023-04-16 23:59:58,301,300,386596750293,1343212502692,4840,False,False,2023-04-16,1
8,2023-04-16 23:59:58,301,300,947673500005,1442321000806,266425750001,True,False,2023-04-16,1
9,2023-04-16 23:59:58,301,2012,368098039,1440974001837,504332,True,False,2023-04-16,1
